In [7]:
!pip install dash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import json




In [8]:
# Initialize the app
app = dash.Dash(__name__)

# Load the dataset
data = pd.read_csv('level_of_study_data.csv')

# Load GeoJSON data
with open('uk_regions_map.geojson', 'r') as file:
    geojson = json.load(file)

# App layout
app.layout = html.Div([
    html.H4('Academic Year Analysis by Region and Level of Study'),
    html.P("Select an Academic Year:"),
    dcc.RadioItems(
        id='year',
        options=[{"label": year, "value": year} for year in data["Academic Year"].unique()],
        value=data["Academic Year"].unique()[0],
        inline=True
    ),
    dcc.Graph(id="choropleth-graph"),
    dcc.Graph(id="pie-chart-graph"),
])

# Callback to update choropleth graph
@app.callback(
    Output("choropleth-graph", "figure"),
    Input("year", "value"))
def update_choropleth(year):
    filtered_data = data[data["Academic Year"] == year]
    aggregated_data = filtered_data.groupby("Region of HE provider")["Number"].sum().reset_index()
    aggregated_data.columns = ["Region", "Number of students"]

    fig = px.choropleth(
        aggregated_data, geojson=geojson, color="Number of students",
        locations="Region", featureidkey="properties.name",
        projection="mercator", range_color=[0, aggregated_data["Number of students"].max()])

    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

# Callback to update pie chart graph
@app.callback(
    Output("pie-chart-graph", "figure"),
    Input("year", "value"),
    Input("choropleth-graph", "clickData"))
def update_pie_chart(year, clickData):
    if clickData is None:
        region = data["Region of HE provider"].unique()[0]
    else:
        region = clickData["points"][0]["location"]

    filtered_data = data[(data["Academic Year"] == year) & (data["Region of HE provider"] == region)]

    fig = px.pie(filtered_data, values='Number', names='Level of study', title=f'Level of Study Distribution in {region}')

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [ ]:

'''import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import json

# Initialize the app
app = dash.Dash(__name__)

# Load the dataset
data = pd.read_csv('level_of_study_data.csv')

# Load GeoJSON data
with open('uk_regions_map.geojson', 'r') as file:
    geojson = json.load(file)

# Extract the centroid coordinates of each region from the GeoJSON
region_centroids = {}
for feature in geojson['features']:
    region_name = feature['properties']['name']
    coordinates = feature['geometry']['coordinates']
    if feature['geometry']['type'] == 'Polygon':
        lon, lat = zip(*coordinates[0])
    elif feature['geometry']['type'] == 'MultiPolygon':
        lon, lat = zip(*coordinates[0][0])
    region_centroids[region_name] = (sum(lon) / len(lon), sum(lat) / len(lat))

# App layout
app.layout = html.Div([
    html.H4('Academic Year Analysis by Region and Level of Study'),
    html.P("Select an Academic Year:"),
    dcc.RadioItems(
        id='year',
        options=[{"label": year, "value": year} for year in data["Academic Year"].unique()],
        value=data["Academic Year"].unique()[0],
        inline=True
    ),
    dcc.Graph(id="choropleth-graph"),
    dcc.Graph(id="pie-chart-graph"),
])

# Callback to update choropleth graph
@app.callback(
    Output("choropleth-graph", "figure"),
    Input("year", "value"))
def update_choropleth(year):
    filtered_data = data[data["Academic Year"] == year]
    aggregated_data = filtered_data.groupby("Region of HE provider")["Number"].sum().reset_index()
    aggregated_data.columns = ["Region", "Number of students"]

    fig = px.choropleth(
        aggregated_data, geojson=geojson, color="Number of students",
        locations="Region", featureidkey="properties.name",
        projection="mercator", range_color=[0, aggregated_data["Number of students"].max()])

    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

# Callback to update pie chart graph
@app.callback(
    Output("pie-chart-graph", "figure"),
    Input("year", "value"),
    Input("choropleth-graph", "clickData"))
def update_pie_chart(year, clickData):
    if clickData is None:
        region = data["Region of HE provider"].unique()[0]
    else:
        region = clickData["points"][0]["location"]

    filtered_data = data[(data["Academic Year"] == year) & (data["Region of HE provider"] == region)]

    fig = go.Figure()

    # Add pie chart
    pie_chart = px.pie(filtered_data, values='Number', names='Level of study')
    fig.add_trace(pie_chart.data[0])

    # Center pie chart on the region's centroid
    centroid = region_centroids[region]
    fig.update_layout(
        annotations=[dict(
            text=f"{region}",
            x=centroid[0],
            y=centroid[1],
            xref='x',
            yref='y',
            showarrow=False,
            font=dict(size=14, color="black")
        )],
        geo=dict(
            scope='europe',
            projection=go.layout.geo.Projection(type='mercator'),
            showland=True,
        )
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)'''


<IPython.core.display.Javascript object>

'import dash\nfrom dash import dcc, html\nfrom dash.dependencies import Input, Output\nimport plotly.express as px\nimport plotly.graph_objects as go\nimport pandas as pd\nimport json\n\n# Initialize the app\napp = dash.Dash(__name__)\n\n# Load the dataset\ndata = pd.read_csv(\'level_of_study_data.csv\')\n\n# Load GeoJSON data\nwith open(\'uk_regions_map.geojson\', \'r\') as file:\n    geojson = json.load(file)\n\n# Extract the centroid coordinates of each region from the GeoJSON\nregion_centroids = {}\nfor feature in geojson[\'features\']:\n    region_name = feature[\'properties\'][\'name\']\n    coordinates = feature[\'geometry\'][\'coordinates\']\n    if feature[\'geometry\'][\'type\'] == \'Polygon\':\n        lon, lat = zip(*coordinates[0])\n    elif feature[\'geometry\'][\'type\'] == \'MultiPolygon\':\n        lon, lat = zip(*coordinates[0][0])\n    region_centroids[region_name] = (sum(lon) / len(lon), sum(lat) / len(lat))\n\n# App layout\napp.layout = html.Div([\n    html.H4(\'Ac

In [ ]:
'''!pip install dash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import json

# Initialize the app
app = dash.Dash(__name__)

# Sample data provided
data = {
    "Academic Year": ["2014/15", "2015/16", "2016/17", "2017/18", "2018/19", "2019/20", "2020/21", "2021/22"],
    "East Midlands": [1253980, 1274705, 1327365, 1379190, 1418105, 1479495, 1539970, 1602235],
    "East of England": [977490, 972705, 997805, 1014345, 1061450, 1143095, 1308285, 1360035],
    "London": [3233450, 3213155, 3286520, 3393810, 3413535, 3495355, 3881845, 4058580],
    "North East": [793725, 801495, 811795, 829890, 847415, 886690, 960405, 997995],
    "North West": [1798060, 1838610, 1873530, 1877075, 1889980, 1930595, 2106600, 2157430],
    "South East": [1932280, 1999225, 2072215, 2098650, 2111055, 2176495, 2290475, 2380180],
    "South West": [1243960, 1263440, 1310735, 1333205, 1359650, 1389180, 1485540, 1584480],
    "West Midlands": [1513945, 1571230, 1609685, 1655940, 1688595, 1747030, 1880500, 1939670],
    "Yorkshire and The Humber": [1535585, 1549325, 1543210, 1546060, 1577000, 1579330, 1709300, 1782655]
}

# Convert data to DataFrame
df = pd.DataFrame(data)

# Load GeoJSON data
with open('uk_regions_map.geojson', 'r') as file:
    geojson = json.load(file)

# Load the level of study data
level_of_study_data = pd.read_csv('/mnt/data/level_of_study_data.csv')

# App layout
app.layout = html.Div([
    html.H4('Academic Year Analysis by Region with Level of Study'),
    html.P("Select an Academic Year:"),
    dcc.RadioItems(
        id='year',
        options=[{"label": year, "value": year} for year in df["Academic Year"]],
        value="2014/15",
        inline=True
    ),
    dcc.Graph(id="graph"),
])

# Callback to update graph
@app.callback(
    Output("graph", "figure"),
    Input("year", "value"))
def display_choropleth(year):
    # Filter data for the selected year
    data_for_year = df[df["Academic Year"] == year].drop("Academic Year", axis=1).T.reset_index()
    data_for_year.columns = ["Region", "Number of students"]

    # Create choropleth figure
    fig = px.choropleth(
        data_for_year, geojson=geojson, color="Number of students",
        locations="Region", featureidkey="properties.name",
        projection="mercator", range_color=[0, data_for_year["Number of students"].max()])

    # Add pie charts to the choropleth
    regions = data_for_year["Region"].unique()
    for region in regions:
        region_data = level_of_study_data[(level_of_study_data["Academic Year"] == year) &
                                          (level_of_study_data["Region of HE provider"] == region)]
        if not region_data.empty:
            fig.add_trace(go.Pie(
                values=region_data["Number"],
                labels=region_data["Level of study"],
                domain=dict(x=[0.1, 0.2], y=[0.8, 0.9]),  # Example domain positioning
                name=region
            ))

    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
'''

'!pip install dash\nimport dash\nfrom dash import dcc, html\nfrom dash.dependencies import Input, Output\nimport plotly.express as px\nimport plotly.graph_objects as go\nimport pandas as pd\nimport json\n\n# Initialize the app\napp = dash.Dash(__name__)\n\n# Sample data provided\ndata = {\n    "Academic Year": ["2014/15", "2015/16", "2016/17", "2017/18", "2018/19", "2019/20", "2020/21", "2021/22"],\n    "East Midlands": [1253980, 1274705, 1327365, 1379190, 1418105, 1479495, 1539970, 1602235],\n    "East of England": [977490, 972705, 997805, 1014345, 1061450, 1143095, 1308285, 1360035],\n    "London": [3233450, 3213155, 3286520, 3393810, 3413535, 3495355, 3881845, 4058580],\n    "North East": [793725, 801495, 811795, 829890, 847415, 886690, 960405, 997995],\n    "North West": [1798060, 1838610, 1873530, 1877075, 1889980, 1930595, 2106600, 2157430],\n    "South East": [1932280, 1999225, 2072215, 2098650, 2111055, 2176495, 2290475, 2380180],\n    "South West": [1243960, 1263440, 1310735, 